This notebook serves to extract the comments out of the two excel files, clean up the text, and place the comments into separate text files within relevant folders.  This is the data structure expected for topic modeling systems.

Folder names will be determined by my understanding of the meaning of these groups.

There aren't any natural IDs for these comments, so I'll need to construct my own file names for these documents. I'm going to make a combination of the page id, type of comment (reply or comment), and a natural number id.  This should make things pretty easy to track down, and I'll reexport the excel file out with these embedded names.

In [129]:
import pandas as pd
from bs4 import BeautifulSoup
import json

In [130]:
def clean_text(html):
    if type(html) != str or len(html.strip()) == 0:
        text = ''
    else:
        soup = BeautifulSoup(html)
        text = soup.get_text()
    return text

In [131]:
vaxxdata = '../NYT data/NYTCommentsWithVaccin-vaccin-VACCIN-Vaxx-vaxx-VAXX.xlsx'
questiondata = '../NYT data/NYTQuestionComments.xlsx'

vaxxtarget = '../documents/vaxx/'
questiontarget = '../documents/questions/'

target = questiontarget

In [139]:
df = pd.read_excel(questiondata, header = None, parse_cols = "A:D")
df.columns = ['short_name', 'type', 'comment_text', 'full_url']

In [140]:
df.describe()

,short_name,type,comment_text,full_url
count,126170,126172,126164,126075
unique,4297,3,125522,4461
top,inside-amazon-wrestling-big-ideas-in-a-bruisin...,comment,Why?,http://www.nytimes.com/2015/08/16/technology/i...
freq,1019,72124,12,1019


In [145]:
js = json.loads(df.to_json(orient = 'records'))
print(len(js))

126172


In [148]:
js[0]

{'short_name': 'outlander-season-1-finale-jamie-rape',
 'type': 'comment',
 'comment_text': 'Unfortunately, rape is treated as a plot device in entertainment. How can we shock? How can we up the ante? Let\'s rape Sansa. We are becoming accustomed to rape. It\'s losing the ability to create horror, unless you happen to be a survivor. Its filmed in soft focus, the emotional impact is lost. Its romanticized. We forget that rape is not an act of sex, but of violence. <br/><br/>THAT is why the scene had to be explicit. That is why we had to see the blood on his thighs and the emptiness in his eyes. <br/><br/>With this extended scene, we see not just the physical horror, but the emotional twisting of the victim\'s psyche as well. For Jamie, the true horror wasn\'t so much the physical assault as it was his cooperation with the rapist.<br/><br/>We still hear people say, "She was asking for it," or "What did you do to make him so angry?" Men are told "You can\'t rape the willing" or "You shoul

In [149]:
allshorts = [r['short_name'] for r in js]

data = {sn: {'comment': [], 'userReply': [], 'reporterReply': []} for sn in allshorts}

for record in js:
    data[ record['short_name'] ][ record['type'] ].append(\
                                                    clean_text(record['comment_text']) )

In [152]:
for sn, types in data.items():
    for thetype, text in types.items():
        if text == []:
            continue
        else:
            for i, comment in enumerate(text):
                filename = target + str(sn).replace('.', '-') + '_' + thetype + '_' + str(i).zfill(4) + '.txt'
                with open(filename, 'w') as fout:
                    fout.write(comment)